#### Step 1: Import the Required Packages

In [2]:
import pandas as pd
import numpy as np
import json
from pandas.io.json import json_normalize
from geopy.geocoders import Nominatim

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans
import folium
import requests

#### step 2: Source the Data required and Understand the data

In [3]:
df1=pd.read_csv('Toronto_Neighborhood.csv')
df2=pd.read_csv('Geospatial_Coordinates.csv')

#### Step 3 : Prepare the Data for the analysis 

In [4]:
df=pd.merge(df1,df2,how='inner',on='Postal Code')
df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494


#### step 4: Analyse the Data 

In [5]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df['Borough'].unique()),
        df.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


In [6]:
df['Borough'].value_counts()

North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
York                 5
East York            5
East Toronto         5
Mississauga          1
Name: Borough, dtype: int64

In [7]:
df['Borough'].unique()

array(['North York', 'Downtown Toronto', 'Etobicoke', 'Scarborough',
       'East York', 'York', 'East Toronto', 'West Toronto',
       'Central Toronto', 'Mississauga'], dtype=object)

In This Notebook, We will try to Cluster Venues based of the Borough 'North York'.We  are taking the subset of the dataset to concentrate on the North York Borough.

In [67]:
df=df[df['Borough']=='North York'].reset_index(drop=True)
df.shape

(24, 5)

##### Step 5: We will get the Latitude and Longitude of The North York  and Visualise it on the map using the folium package.

In [65]:
address='North York'
geolocator=Nominatim(user_agent='Tto_explorer')
location=geolocator.geocode(address)
lat=location.latitude
long=location.longitude
print('Latitude is :{},longitude is :{},\nAddress as per geolocator:{}'.format(lat,long,location))

Latitude is :43.7543263,longitude is :-79.44911696639593,
Address as per geolocator:North York, Toronto, Golden Horseshoe, Ontario, Canada


In [69]:
map_Ny=folium.Map(location=[lat,long],zoom_start=10)

In [70]:
map_Ny

In [71]:
for lat,long,name,Neigh in zip(df['Latitude'],df['Longitude'],df['Borough'],df['Neighborhood']):
    label = '{}, {}'.format(Neigh,name)
    label = folium.Popup(label, parse_html=True)
    
    folium.CircleMarker(
     [lat, long],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186aa',
        fill_opacity=0.7,
        parse_html=True).add_to(map_Ny)
map_Ny    

#### Step 6: We are Using the Foursquare API to get the nearby venues.We need to provide Required details to make an URL for exploring the venues.

In [13]:
CLIENT_ID = 'UAUWWOD1AVU1ATUDRVZMLXM0J3PYB3DMMKFA05UKCLLD1BDM' # your Foursquare ID
CLIENT_SECRET = 'MPXH2IXXNOBIPL1SJ2EQ1SPLMUMSTLZITVEYL3XOQGNG0BLP' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: UAUWWOD1AVU1ATUDRVZMLXM0J3PYB3DMMKFA05UKCLLD1BDM
CLIENT_SECRET:MPXH2IXXNOBIPL1SJ2EQ1SPLMUMSTLZITVEYL3XOQGNG0BLP


##### step 7: We create the URL with specifications like LIMIT of  venues and radius from the specified location.In this case we are providng the coordinates of "NorthYork" Borough

#### Now lets get the top 100 venues near Parkwoods  in the North York Borough

In [23]:
df.loc[0,'Neighborhood']

'Parkwoods'

In [221]:
address='Parkwoods'
geolocator=Nominatim(user_agent='pw_exp')
location=geolocator.geocode(address)
lat=location.latitude
long=location.longitude
print('Latitude is :{},longitude is :{},\nAddress as per geolocator:{}'.format(lat,long,location))

Latitude is :37.8567738,longitude is :-122.22068778004532,
Address as per geolocator:Parkwoods Condominiums, Oakland, Alameda County, California, United States of America


In [224]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    lat, 
    long, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=UAUWWOD1AVU1ATUDRVZMLXM0J3PYB3DMMKFA05UKCLLD1BDM&client_secret=MPXH2IXXNOBIPL1SJ2EQ1SPLMUMSTLZITVEYL3XOQGNG0BLP&v=20180604&ll=37.8567738,-122.22068778004532&radius=500&limit=100'

#### step 8 : We have created an explore link ,Which  gives an output in Json format with the  Top 100 venues nearby .We can explore the output of the same below : 

In [226]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e9df4120de0d9002833157a'},
 'response': {'headerLocation': 'Claremont Elmwood',
  'headerFullLocation': 'Claremont Elmwood, Oakland',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 6,
  'suggestedBounds': {'ne': {'lat': 37.8612738045, 'lng': -122.2149989473819},
   'sw': {'lat': 37.8522737955, 'lng': -122.22637661270875}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b4cad57f964a520e3b926e3',
       'name': 'Parkwoods Gym',
       'location': {'address': '176 Caldecott Ln',
        'lat': 37.85647844858324,
        'lng': -122.22077914411892,
        'labeledLatLngs': [{'label': 'display',
          'lat': 37.85647844858324,
          'lng': -122.22077914411892}],
        'distance': 33,
        'postalCode': '9

From the Foursquare lab in the previous module, we know that all the information is in the *items* key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.

In [227]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
#filtered_columns = ['venue.name', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,Parkwoods Gym,Gym,37.856478,-122.220779
1,Wild Turkey Hill,Trail,37.856666,-122.219858
2,Parkwoods Pool,Pool,37.856500,-122.220740
3,Parkwoods Hot Tub,Water Park,37.856443,-122.220774
4,Caldecott Tunnel,Tunnel,37.854929,-122.216535
5,The spot,Scenic Lookout,37.857198,-122.225146


In [228]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

6 venues were returned by Foursquare.


##### Step  9:We get the vnues of all the Neighbourhoods in the 'North York'  Borough

In [80]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [81]:


df_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )



Parkwoods
Victoria Village
Lawrence Manor / Lawrence Heights
Don Mills
Glencairn
Don Mills
Hillcrest Village
Bathurst Manor / Wilson Heights / Downsview North
Fairview / Henry Farm / Oriole
Northwood Park / York University
Bayview Village
Downsview
York Mills / Silver Hills
Downsview
North Park / Maple Leaf Park / Upwood Park
Humber Summit
Willowdale / Newtonbrook
Downsview
Bedford Park / Lawrence Manor East
Humberlea / Emery
Willowdale
Downsview
York Mills West
Willowdale


In [119]:
df_venues.head()


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


In [133]:
df_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Bathurst Manor / Wilson Heights / Downsview North,22,22,22,22,22,22
Bayview Village,4,4,4,4,4,4
Bedford Park / Lawrence Manor East,24,24,24,24,24,24
Don Mills,26,26,26,26,26,26
Downsview,17,17,17,17,17,17
Fairview / Henry Farm / Oriole,70,70,70,70,70,70
Glencairn,4,4,4,4,4,4
Hillcrest Village,4,4,4,4,4,4
Humber Summit,2,2,2,2,2,2


##### Step 9 :We will One hot enocode all the Venue categories located of all the venues which are identified to be able to  profile the venues according to the presence of Category

In [229]:
# one hot encoding
df_onehot = pd.get_dummies(df_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
df_onehot['Neighborhood'] = df_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [df_onehot.columns[-1]] + list(df_onehot.columns[:-1])
df_onehot = df_onehot[fixed_columns]

df_onehot.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,...,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Women's Store
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [40]:
df_onehot.shape

(249, 105)

In [232]:
df_grouped = df_onehot.groupby('Neighborhood').mean().reset_index()
df_grouped

,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,...,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Women's Store
0,Bathurst Manor / Wilson Heights / Downsview North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,...,0.045455,0.045455,0.000000,0.000000,0.000000,0.000000,0.000000,0.045455,0.000000,0.000000
1,Bayview Village,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Bedford Park / Lawrence Manor East,0.000000,0.000000,0.041667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.041667,0.000000,0.041667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Don Mills,0.000000,0.000000,0.000000,0.000000,0.076923,0.038462,0.000000,0.000000,0.000000,...,0.038462,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Downsview,0.000000,0.058824,0.000000,0.000000,0.000000,0.058824,0.000000,0.058824,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Fairview / Henry Farm / Oriole,0.000000,0.000000,0.014286,0.000000,0.014286,0.000000,0.028571,0.028571,0.014286,...,0.000000,0.000000,0.014286,0.000000,0.014286,0.014286,0.014286,0.000000,0.000000,0.042857
6,Glencairn,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Hillcrest Village,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Humber Summit,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Humberlea / Emery,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [43]:
df_grouped.shape

(19, 105)

In [177]:
num_top_venues = 5

for hood in df_grouped['Neighborhood']:
    print("----"+hood+"----")
    
    temp = df_grouped[df_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bathurst Manor / Wilson Heights / Downsview North----
                 venue  freq
0          Coffee Shop  0.09
1                 Bank  0.09
2       Sandwich Place  0.05
3  Fried Chicken Joint  0.05
4        Deli / Bodega  0.05


----Bayview Village----
                 venue  freq
0   Chinese Restaurant  0.25
1                 Bank  0.25
2                 Café  0.25
3  Japanese Restaurant  0.25
4    Accessories Store  0.00


----Bedford Park / Lawrence Manor East----
                venue  freq
0      Sandwich Place  0.08
1  Italian Restaurant  0.08
2         Coffee Shop  0.08
3          Restaurant  0.08
4         Pizza Place  0.04


----Don Mills----
                 venue  freq
0          Coffee Shop  0.08
1     Asian Restaurant  0.08
2           Beer Store  0.08
3           Restaurant  0.08
4  Japanese Restaurant  0.08


----Downsview----
            venue  freq
0   Grocery Store  0.18
1            Park  0.12
2           Hotel  0.06
3  Discount Store  0.06
4    Liquor Store  0.

#### Step 10: We create a data frame with the top 10 most common Venue category for Each Neighbourhood

In [46]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [233]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue Category'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue Category'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = df_grouped['Neighborhood']

for ind in np.arange(df_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(df_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue Category,2nd Most Common Venue Category,3rd Most Common Venue Category,4th Most Common Venue Category,5th Most Common Venue Category,6th Most Common Venue Category,7th Most Common Venue Category,8th Most Common Venue Category,9th Most Common Venue Category,10th Most Common Venue Category
0,Bathurst Manor / Wilson Heights / Downsview North,Coffee Shop,Bank,Gas Station,Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Diner,Deli / Bodega,Ice Cream Shop,Convenience Store
1,Bayview Village,Chinese Restaurant,Café,Bank,Japanese Restaurant,Women's Store,Discount Store,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
2,Bedford Park / Lawrence Manor East,Sandwich Place,Coffee Shop,Italian Restaurant,Restaurant,Fast Food Restaurant,Comfort Food Restaurant,Café,Butcher,Indian Restaurant,Pharmacy
3,Don Mills,Beer Store,Coffee Shop,Japanese Restaurant,Asian Restaurant,Restaurant,Gym,Chinese Restaurant,Clothing Store,Caribbean Restaurant,Café
4,Downsview,Grocery Store,Park,Construction & Landscaping,Bank,Hotel,Home Service,Gym / Fitness Center,Shopping Mall,Baseball Field,Athletics & Sports


#### Step 11: We are fitting  kmeans with 3 clusters for all the Neighbourhoods across all the Venue Categories

In [205]:
# set number of clusters
kclusters = 3

grouped_clustering = df_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0])

#### Step 12:We add cluster labels to dataframe

In [234]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

df_merged = df_venues

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
df_merged = df_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

df_merged.head() # check the last columns!

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue Category,2nd Most Common Venue Category,3rd Most Common Venue Category,4th Most Common Venue Category,5th Most Common Venue Category,6th Most Common Venue Category,7th Most Common Venue Category,8th Most Common Venue Category,9th Most Common Venue Category,10th Most Common Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park,0,Park,Food & Drink Shop,Diner,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop,0,Park,Food & Drink Shop,Diner,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena,0,Coffee Shop,Pizza Place,Hockey Arena,Portuguese Restaurant,Diner,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop
3,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop,0,Coffee Shop,Pizza Place,Hockey Arena,Portuguese Restaurant,Diner,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant,0,Coffee Shop,Pizza Place,Hockey Arena,Portuguese Restaurant,Diner,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop


#### We can Visualise the Clusters on the map

In [214]:
# create map
map_clusters = folium.Map(location=[lat, long], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_merged['Latitude'], df_merged['Longitude'], df_merged['Neighborhood'], df_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.8).add_to(map_clusters)
       
map_clusters

### Below Given are the Clusters Identified  With  a significant difference based on the availability of Venue Categories in the Borough of  'North York'

In [56]:
df_merged.loc[df_merged['Cluster Labels'] == 0, df_merged.columns[[1] + list(range(5, df_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,0,Park,Food & Drink Shop,Diner,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
1,North York,0,Coffee Shop,Pizza Place,Hockey Arena,Portuguese Restaurant,Diner,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop
2,North York,0,Accessories Store,Clothing Store,Sporting Goods Shop,Arts & Crafts Store,Boutique,Coffee Shop,Event Space,Furniture / Home Store,Vietnamese Restaurant,Miscellaneous Shop
3,North York,0,Beer Store,Coffee Shop,Japanese Restaurant,Asian Restaurant,Restaurant,Gym,Chinese Restaurant,Clothing Store,Caribbean Restaurant,Café
4,North York,0,Asian Restaurant,Pizza Place,Pub,Japanese Restaurant,Women's Store,Dim Sum Restaurant,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store
5,North York,0,Beer Store,Coffee Shop,Japanese Restaurant,Asian Restaurant,Restaurant,Gym,Chinese Restaurant,Clothing Store,Caribbean Restaurant,Café
6,North York,0,Golf Course,Mediterranean Restaurant,Pool,Dog Run,Women's Store,Dim Sum Restaurant,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store
7,North York,0,Coffee Shop,Bank,Gas Station,Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Diner,Deli / Bodega,Ice Cream Shop,Convenience Store
8,North York,0,Clothing Store,Coffee Shop,Fast Food Restaurant,Women's Store,Restaurant,Sporting Goods Shop,Bus Station,Japanese Restaurant,Bank,Convenience Store
9,North York,0,Coffee Shop,Massage Studio,Caribbean Restaurant,Falafel Restaurant,Bar,Miscellaneous Shop,Dim Sum Restaurant,Comfort Food Restaurant,Concert Hall,Construction & Landscaping


In [57]:
df_merged.loc[df_merged['Cluster Labels'] == 1, df_merged.columns[[1] + list(range(5, df_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,North York,1,Piano Bar,Women's Store,Clothing Store,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store


In [58]:
df_merged.loc[df_merged['Cluster Labels'] == 2, df_merged.columns[[1] + list(range(5, df_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,North York,2,Cafeteria,Women's Store,Coffee Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop
